In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
inter_df = pd.read_csv('./국토교통부_건축물대장_표제부/bdinfo_baseline_maindata_inter_checkpoint_2.csv')
inter_df['사용승인_일'] = inter_df['사용승인_일'].astype('Int64')

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
inter_df.head()

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인_년도,주소별_기준년월_총세대수합
0,11440-21639,집합,표제부,0,177.62,105.300,59.28,393.130,385.210,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,12.00,4,1,0,0,0,0.0,393.130,0,0.0,0,0.0,0,0.00,4,46.0,20040323,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16,4,일반,20040323.0,20040323.0,20101230,20170209,2004.0,4
1,11590-100182663,집합,표제부,1,389.48,232.420,59.67,947.740,659.890,169.43,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,11.26,4,1,0,0,0,0.0,947.740,0,0.0,0,0.0,8,287.85,2,11.5,20080826,NaN,0.0,0,NaN,0,NaN,0,20110119,201412,서울특별시 동작구 상도동 302-28,10,일반,20080826.0,20080826.0,20110119,20190530,2008.0,10
2,11320-100182241,집합,표제부,0,208.00,81.085,38.98,268.165,268.165,128.93,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,10.40,4,0,0,0,0,0.0,268.165,0,0.0,0,0.0,3,0.00,1,0.0,20080721,NaN,0.0,0,NaN,0,NaN,0,20110202,201412,서울특별시 도봉구 쌍문동 380-24,4,일반,20080721.0,20080721.0,20110202,20190530,2008.0,4
3,11440-32174,집합,표제부,0,314.00,180.380,57.45,627.880,627.880,199.96,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,평스라브,13.30,5,0,0,0,0,0.0,627.880,0,0.0,0,0.0,7,80.50,1,11.5,20050901,NaN,0.0,0,NaN,0,NaN,0,20110125,201412,서울특별시 마포구 성산동 200-260,8,일반,20050901.0,20050901.0,20110125,20171205,2005.0,8
4,11740-100179671,집합,표제부,0,0.00,564.010,0.00,6401.190,6401.190,0.00,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트,45.60,14,0,3,0,0,0.0,6401.190,0,0.0,0,0.0,0,0.00,0,0.0,20090311,NaN,0.0,0,NaN,0,NaN,0,20110122,201412,서울특별시 강동구 강일동 674-0,78,일반,20090311.0,20090311.0,20100918,20200428,2009.0,1001


In [4]:
inter_df['지붕'].value_counts()

(철근)콘크리트    834915
기타지붕         37106
기와           19752
슬레이트          1835
Name: 지붕, dtype: int64

In [5]:
inter_df['기준년월_최초사용승인일'].fillna(inter_df['주소별_최초생성일자'], inplace=True)
inter_df['기준년월_최초사용승인일'].isna().sum()

0

In [6]:
roof_df = inter_df[['지번주소', '기준년월', '지붕', '기준년월_최초사용승인일']]
roof_df.head()

,지번주소,기준년월,지붕,기준년월_최초사용승인일
0,서울특별시 마포구 서교동 484-16,201412,(철근)콘크리트,20040323.0
1,서울특별시 동작구 상도동 302-28,201412,(철근)콘크리트,20080826.0
2,서울특별시 도봉구 쌍문동 380-24,201412,(철근)콘크리트,20080721.0
3,서울특별시 마포구 성산동 200-260,201412,(철근)콘크리트,20050901.0
4,서울특별시 강동구 강일동 674-0,201412,(철근)콘크리트,20090311.0


In [7]:
roof_df['count'] = roof_df[['지번주소', '기준년월', '지붕']].groupby(['지번주소', '기준년월']).transform('count')['지붕']
roof_df.head()

,지번주소,기준년월,지붕,기준년월_최초사용승인일,count
0,서울특별시 마포구 서교동 484-16,201412,(철근)콘크리트,20040323.0,1
1,서울특별시 동작구 상도동 302-28,201412,(철근)콘크리트,20080826.0,1
2,서울특별시 도봉구 쌍문동 380-24,201412,(철근)콘크리트,20080721.0,1
3,서울특별시 마포구 성산동 200-260,201412,(철근)콘크리트,20050901.0,1
4,서울특별시 강동구 강일동 674-0,201412,(철근)콘크리트,20090311.0,14


In [9]:
roof_df = roof_df.sort_values(['지번주소', '기준년월']).reset_index(drop=True)
roof_df.head(20)

,지번주소,기준년월,지붕,기준년월_최초사용승인일,count
0,서울특별시 강남구 개포동 1163-4,201412,(철근)콘크리트,20130307.0,1
1,서울특별시 강남구 개포동 1163-4,201512,(철근)콘크리트,20130307.0,1
2,서울특별시 강남구 개포동 1163-4,201612,(철근)콘크리트,20130307.0,1
3,서울특별시 강남구 개포동 1163-4,201712,(철근)콘크리트,20130307.0,1
4,서울특별시 강남구 개포동 1163-4,201812,(철근)콘크리트,20130307.0,1
5,서울특별시 강남구 개포동 1163-4,201912,(철근)콘크리트,20130307.0,1
6,서울특별시 강남구 개포동 1163-4,202011,(철근)콘크리트,20130307.0,1
7,서울특별시 강남구 개포동 1164-0,201412,(철근)콘크리트,19910327.0,1
8,서울특별시 강남구 개포동 1164-0,201512,(철근)콘크리트,19910327.0,1
9,서울특별시 강남구 개포동 1164-0,201612,(철근)콘크리트,19910327.0,1


In [12]:
print(roof_df.shape)
roof_df.dropna(inplace=True)
roof_df.shape

(894005, 9)


(893608, 9)

In [13]:
for roof in roof_df['지붕'].unique():
    roof_df['지붕_'+roof] = roof_df['지붕'].apply(lambda x: 1 if x == roof else 0)

In [14]:
for roof in tqdm(roof_df['지붕'].unique()):
    roof_df['지붕_'+roof+'_sum'] = roof_df[['지번주소', '기준년월', '지붕_'+roof]].groupby(['지번주소', '기준년월']).transform('sum')['지붕_'+roof]

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.63it/s]


In [15]:
for roof in tqdm(roof_df['지붕'].unique()):
    roof_df['지붕_'+roof+'_fraction'] = roof_df['지붕_'+roof+'_sum'] / roof_df['count']

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 148.11it/s]


In [17]:
roof_df.head(20)

,지번주소,기준년월,지붕,기준년월_최초사용승인일,count,지붕_(철근)콘크리트,지붕_기타지붕,지붕_기와,지붕_슬레이트,지붕_(철근)콘크리트_sum,지붕_기타지붕_sum,지붕_기와_sum,지붕_슬레이트_sum,지붕_(철근)콘크리트_fraction,지붕_기타지붕_fraction,지붕_기와_fraction,지붕_슬레이트_fraction
0,서울특별시 강남구 개포동 1163-4,201412,(철근)콘크리트,20130307.0,1,1,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0
1,서울특별시 강남구 개포동 1163-4,201512,(철근)콘크리트,20130307.0,1,1,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0
2,서울특별시 강남구 개포동 1163-4,201612,(철근)콘크리트,20130307.0,1,1,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0
3,서울특별시 강남구 개포동 1163-4,201712,(철근)콘크리트,20130307.0,1,1,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0
4,서울특별시 강남구 개포동 1163-4,201812,(철근)콘크리트,20130307.0,1,1,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0
5,서울특별시 강남구 개포동 1163-4,201912,(철근)콘크리트,20130307.0,1,1,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0
6,서울특별시 강남구 개포동 1163-4,202011,(철근)콘크리트,20130307.0,1,1,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0
7,서울특별시 강남구 개포동 1164-0,201412,(철근)콘크리트,19910327.0,1,1,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0
8,서울특별시 강남구 개포동 1164-0,201512,(철근)콘크리트,19910327.0,1,1,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0
9,서울특별시 강남구 개포동 1164-0,201612,(철근)콘크리트,19910327.0,1,1,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0


In [18]:
cols_to_keep = [col for col in roof_df.columns if 'fraction' in col]
cols_to_keep

['지붕_(철근)콘크리트_fraction',
 '지붕_기타지붕_fraction',
 '지붕_기와_fraction',
 '지붕_슬레이트_fraction']

In [19]:
export_df = roof_df[['지번주소', '기준년월', '기준년월_최초사용승인일'] + cols_to_keep]
export_df.head()

,지번주소,기준년월,기준년월_최초사용승인일,지붕_(철근)콘크리트_fraction,지붕_기타지붕_fraction,지붕_기와_fraction,지붕_슬레이트_fraction
0,서울특별시 강남구 개포동 1163-4,201412,20130307.0,1.0,0.0,0.0,0.0
1,서울특별시 강남구 개포동 1163-4,201512,20130307.0,1.0,0.0,0.0,0.0
2,서울특별시 강남구 개포동 1163-4,201612,20130307.0,1.0,0.0,0.0,0.0
3,서울특별시 강남구 개포동 1163-4,201712,20130307.0,1.0,0.0,0.0,0.0
4,서울특별시 강남구 개포동 1163-4,201812,20130307.0,1.0,0.0,0.0,0.0


In [20]:
print(export_df.shape)
export_df.drop_duplicates(inplace=True)
export_df.shape

(893608, 7)


(732337, 7)